In [1]:
import pandas as pd
import wanikani_api.client as client
from wanakana import to_kana, is_romaji

In [2]:
api_key = "eae4dc26-0138-4565-a8c6-4d6761a69248"
wk_api = client.Client(api_key)

In [50]:
levels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
vocabulary = wk_api.subjects(types="vocabulary", levels = levels, fetch_all=True)

vocab = vocabulary[0]
#dir(vocab)

In [61]:
dir(vocabulary[0].pronunciation_audios[0])
vocabulary[0].pronunciation_audios[0].url

'https://files.wanikani.com/xzymh7gye9b6p59oqtgtahxwcfw7'

In [4]:
vocab.characters                      # returns kanji
vocab.auxiliary_meanings[0].meaning   # returns auxiliary meaning
vocab.meanings[0].meaning             # returns primary meaning
vocab.readings[0].reading             # returns hiragana reading
vocab.context_sentences[0].english    # returns context setences in english
vocab.context_sentences[0].japanese   # returns context setences in japanese

'一人でカラオケにいきました。'

In [38]:
columns = ['index', 'word', 'word_with_reading', 'defintion', 'example_sentence', 'sentence_with_reading', 'setence_translation', 'Kanji', 'v1', 'word_audio', 'setence_audio', 'level']

In [39]:
data = []

for index, vocab in enumerate(vocabulary):
    info = [index+1001, vocab.characters, vocab.readings[0].reading, vocab.meanings[0].meaning, vocab.context_sentences[0].japanese, None, vocab.context_sentences[0].english, None, None, None, None, vocab.level]

    data.append(info)

In [40]:
df = pd.DataFrame(data, columns=columns)
df

,index,word,word_with_reading,defintion,example_sentence,sentence_with_reading,setence_translation,Kanji,v1,word_audio,setence_audio,level
0,1001,一,いち,One,このレモン、一キログラム！？大きいですね！,None,This lemon weighs a kilogram!? That's big!,None,None,None,None,1
1,1002,一つ,ひとつ,One Thing,レモン、一つ下さい。,None,"A lemon, please.",None,None,None,None,1
2,1003,七,なな,Seven,コウイチのラッキーナンバーは七です。,None,Koichi's lucky number is seven.,None,None,None,None,1
3,1004,七つ,ななつ,Seven Things,そのクッキー、七つ下さい。,None,"Seven of those cookies, please.",None,None,None,None,1
4,1005,九,きゅう,Nine,ガソリンスタンドまで九マイルです。,None,It is nine miles to the gas station.,None,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1050,2051,十分,じゅうぶん,Enough,あなたは十分がんばりました！,None,You have worked hard enough!,None,None,None,None,6
1051,2052,一ヶ月,いっかげつ,One Month,クリスマスまであと一ヶ月ですね！,None,It's one more month until Christmas!,None,None,None,None,9
1052,2053,〜ヶ月,かげつ,Counter For Months,日本の学校の夏休みは一ヶ月しかありません。,None,Japanese schools only have one month of summer...,None,None,None,None,9
1053,2054,十分間,じゅっぷんかん,Ten Minutes,ここで十分間休みましょう。,None,Let's rest here for ten minutes.,None,None,None,None,9


In [44]:
from sudachipy import tokenizer
from sudachipy import dictionary
from wanakana import to_hiragana, is_japanese, is_katakana, is_hiragana, is_kanji
import string

tokenizer_obj = dictionary.Dictionary(dict_type="small").create()

# KANJI_READING_MAPPING = {
#     '私': '私[わたし]',
#     '貴女': '貴女[あなた]',
#     '何': '何[なに]',
#     '外宇宙': '外宇宙[がいうちゅう]',
#     '異星人': '異星人[いせいじん]',
#     '優那': '優那[ゆうな]',
#     '菜々美': '菜々美[ななみ]'
# }

JAPANESE_PUNCTUATION = '　〜！？。、（）：「」『』０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'

SPECIAL_CHARACTERS = '〜'

def is_japanese_extended(text):
    return is_japanese(text) and text not in string.punctuation and text not in JAPANESE_PUNCTUATION

def to_anki_format(index, kanji, reading):
    return '{}{}[{}]'.format(' ' if index > 0 else '', kanji, reading) 

def add_furigana(text):
    tokens = [m for m in tokenizer_obj.tokenize(text, tokenizer.Tokenizer.SplitMode.C)]
    parsed = ''
    token_indexes_to_skip = []
    for index, token in enumerate(tokens):   
        if index in token_indexes_to_skip:
          continue
        to_parse = is_japanese_extended(token.surface()) and not is_katakana(token.surface()) and not is_hiragana(token.surface())
        if to_parse:
            if token.surface()[-1] in SPECIAL_CHARACTERS:
                parsed += add_furigana(token.surface()[:-1]) + token.surface()[-1]
            else:
                if index > 0:
                    parsed += ' '
                reading = to_hiragana(token.reading_form())
                if token.surface() in KANJI_READING_MAPPING:
                    parsed += KANJI_READING_MAPPING[token.surface()]
                elif index < len(tokens)-1 and token.surface() + tokens[index+1].surface() in KANJI_READING_MAPPING:
                    parsed += KANJI_READING_MAPPING[tokens[index].surface() + tokens[index+1].surface()]
                    token_indexes_to_skip.append(index+1)
                else:
                    surface_index = 0
                    reading_index = 0
                    while len(token.surface()) > surface_index:
                        if is_hiragana(token.surface()[surface_index]) or is_katakana(token.surface()[surface_index]):
                            parsed += token.surface()[surface_index]
                            reading_index += 1
                            surface_index += 1
                        else:
                            next_index = -1
                            for token_index in range(surface_index, len(token.surface())):
                                if is_hiragana(token.surface()[token_index]) or is_katakana(token.surface()[token_index]):
                                    next_index = token_index
                                    break
                            if next_index < 0:
                                parsed += to_anki_format(
                                  index=surface_index, 
                                  kanji=token.surface()[surface_index:], reading=reading[reading_index:])
                                break
                            else:
                                reading_index_tail = reading_index
                                while reading[reading_index_tail] != token.surface()[next_index] or (reading_index_tail < len(reading)-1 and reading[reading_index_tail] == reading[reading_index_tail+1]):
                                    reading_index_tail += 1
                                parsed += to_anki_format(
                                  index=surface_index, 
                                  kanji=token.surface()[surface_index:next_index], reading=reading[reading_index:reading_index_tail])
                                reading_index = reading_index_tail
                            reading_length = next_index - surface_index
                            if reading_length > 0:
                                surface_index += reading_length
                            else:
                                break
        else:
            parsed += token.surface()
    return parsed

# test = ['私', '優那', 'Japanese', '可愛い','いい加減', '喧嘩', '咲く', '入り口', '兎に角', '取り逃す', 'とり戻す', '絞り込んだ', 'この先に多数の機械生命体反応を確認', '広いな……この廃墟。']
# for s in test:
#     print(add_furigana(s))

/var/folders/vt/st5d0xqj4vn9bjz3rgc4jhqc0000gn/T/ipykernel_25716/688407333.py:6: DeprecationWarning: Parameter dict_type of Dictionary() is deprecated, use dict instead
  tokenizer_obj = dictionary.Dictionary(dict_type="small").create()


In [48]:
df['word_with_reading'] = df['word'].apply(add_furigana)
df['sentence_with_reading'] = df['example_sentence'].apply(add_furigana)
df['Kanji'] = ~df['word'].apply(is_katakana)
df

,index,word,word_with_reading,defintion,example_sentence,sentence_with_reading,setence_translation,Kanji,v1,word_audio,setence_audio,level
0,1001,一,一[いち],One,このレモン、一キログラム！？大きいですね！,このレモン、 一[いち]キログラム！？ 大[おお]きいですね！,This lemon weighs a kilogram!? That's big!,True,None,None,None,1
1,1002,一つ,一[ひと]つ,One Thing,レモン、一つ下さい。,レモン、 一[ひと]つ 下[くだ]さい。,"A lemon, please.",True,None,None,None,1
2,1003,七,七[なな],Seven,コウイチのラッキーナンバーは七です。,コウイチのラッキーナンバーは 七[なな]です。,Koichi's lucky number is seven.,True,None,None,None,1
3,1004,七つ,七[なな]つ,Seven Things,そのクッキー、七つ下さい。,そのクッキー、 七[なな]つ 下[くだ]さい。,"Seven of those cookies, please.",True,None,None,None,1
4,1005,九,九[きゅう],Nine,ガソリンスタンドまで九マイルです。,ガソリンスタンドまで 九[きゅう]マイルです。,It is nine miles to the gas station.,True,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1050,2051,十分,十[じゅう] 分[ふん],Enough,あなたは十分がんばりました！,あなたは 十分[じゅうぶん]がんばりました！,You have worked hard enough!,True,None,None,None,6
1051,2052,一ヶ月,一[いち] ヶ 月[げつ],One Month,クリスマスまであと一ヶ月ですね！,クリスマスまであと 一[いち] ヶ 月[げつ]ですね！,It's one more month until Christmas!,True,None,None,None,9
1052,2053,〜ヶ月,〜 ヶ 月[げつ],Counter For Months,日本の学校の夏休みは一ヶ月しかありません。,日本[にっぽん]の 学校[がっこう]の 夏休[なつやす]みは 一[いち] ヶ 月[げつ]しか...,Japanese schools only have one month of summer...,True,None,None,None,9
1053,2054,十分間,十[じゅう] 分間[ふんかん],Ten Minutes,ここで十分間休みましょう。,ここで 十[じゅう] 分間[ふんかん] 休[やす]みましょう。,Let's rest here for ten minutes.,True,None,None,None,9


In [49]:
df.to_csv('data.csv', encoding="utf-8", index=False, header=False)

In [64]:
word = '法隆寺はいつできましたか'

print(add_furigana(word))

法隆[ほうりゅう] 寺[じ]はいつできましたか


In [ ]:
# assignments = wk_api.assignments(subject_types="vocabulary", unlocked=True, fetch_all=True)

In [ ]:
# data = []

# for vocab in assignments:
#     vocab = vocab.subject
#     info = [vocab.characters, vocab.readings[0].reading, vocab.meanings[0].meaning, vocab.context_sentences[0].japanese, vocab.context_sentences[0].english]

#     data.append(info)

In [ ]:
# vocab = assignments[0].subject

In [ ]:
# columns = ['vocabulary', 'vocab_furigana', 'meaning', 'context_sentence', 'contenxt_setence_meaning']

In [ ]:
# data = []

# info = [vocab.characters, vocab.readings[0].reading, vocab.meanings[0].meaning, vocab.context_sentences[0].japanese, vocab.context_sentences[0].english]

# data.append(info)

In [ ]:
# df = pd.DataFrame(data, columns=columns)
# df

In [ ]:
# id = assignments[0].subject.id
# id

In [ ]:
# vocabulary = wk_api.subjects(types="vocabulary", ids = [2504], fetch_all=True)

In [ ]:
# vocab = vocabulary[0]

In [ ]:
# data = []

# info = [vocab.characters, vocab.readings[0].reading, vocab.meanings[0].meaning, vocab.context_sentences[0].japanese, vocab.context_sentences[0].english]

# data.append(info)

In [ ]:
# df = pd.DataFrame(data, columns=columns)
# df

In [ ]:
# assignments = wk_api.assignments(subject_types="vocabulary", unlocked=True, fetch_all=True)

In [ ]:
# dir(assignments[0].subject.id)
# assignments[0].subject.id

In [ ]:
# assignments[989].subject.id